In [55]:
import os
from river import datasets
from river import ensemble
from confluent_kafka import Producer,Consumer
import certifi
import time
import json

In [56]:
user= os.environ['kafka_username']
password= os.environ['kafka_password']
bsts= os.environ['kafka_bootstrap_servers']

In [57]:
conf = {'bootstrap.servers': bsts,
            'sasl.mechanism': 'PLAIN',
            'security.protocol': 'SASL_SSL',
            'ssl.ca.location': certifi.where(),
            'sasl.username': user,
            'sasl.password': password,
            'batch.num.messages': 4000,
            'linger.ms': 100,
            'client.id': 'producer-icde-2023'}
producer = Producer(conf)    

In [78]:
max_size=5000
dataset = datasets.MaliciousURL()
data = dataset.take(max_size)

In [79]:
feature_topic = 'features_v4'
flush_size=600
end=0
cnt = 0
st = time.time()
abs_st = time.time()
for f, y in data:
    cnt = cnt + 1    
    d = {}
    d['f']=f
    d['y']=str(y).lower()
    d['st']=time.time()  
            
    v= json.dumps(d).encode('utf-8')
    try:
        producer.produce(feature_topic, value=v, key=str(cnt))
        if(cnt%flush_size==0):
            print(f'Queue full, flushing {cnt}')
            producer.flush()
            time.sleep(1.2)
    except:
      print(f'Queue full, flushing {cnt}')
      producer.flush()
      end = time.time()
      print(f'flushing count - {cnt}, time taken in seconds- {end-st} ')        
        
      producer.produce(feature_topic, value=v, key=str(cnt))
 
producer.flush()
end = time.time()
print(f'final flushing count - {cnt}, time taken in seconds- {end-abs_st} ')        

Queue full, flushing 600
Queue full, flushing 1200
Queue full, flushing 1800
Queue full, flushing 2400
Queue full, flushing 3000
Queue full, flushing 3600
Queue full, flushing 4200
Queue full, flushing 4800
final flushing count - 5000, time taken in seconds- 11.601880550384521 
